**Imports**

In [195]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import pickle
import missingno as msno
from scipy import stats
from datetime import date
from statistics import mean
from collections import Counter, OrderedDict
import import_ipynb
import UTILS as utils
from time import sleep

import FundamentalAnalysis as fa

In [2]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [3]:
api_key = "Here would go my personal API key"

------------

**Load data**

*Files with data from year with displacement -X*

In [ ]:
file1 = 'DATA_ttm/S_P500_data_ttm_Xyeardisp/S_P500_dataframe_with_ratios_ttm_Xyeardisp.csv'
file2 = 'DATA_ttm/S_P500_data_ttm_Xyeardisp/S_P500_companies_financials_data_ttm_Xyeardisp.pickle'

In [275]:
data_loaded = utils.data_loading(file1, file2)

In [276]:
df = data_loaded[0]
dict_companies = data_loaded[1]

In [ ]:
df.head(2)

---------

**Add sectors and industries info to the dataset**

In [277]:
sectors_df, industries_df = utils.get_company_sectors_and_industries(df, dict_companies)

In [278]:
if set(['sector','industry']).issubset(df.columns) == False:
    df.insert(0, "sector", sectors_df.sector)
    df.insert(1, "industry", industries_df.industry)

-------------------

**Compute the intrinsic value**

For GGM:
<br>
<br>
**Intrinsic Value = EDPS / (CCE-DGR)** <br>
where:<br>
EDPS = Expected Dividend per Share for the next year = Dividends * (1 + DGR)<br>
CCE = Cost of Capital Equity<br>
DGR = Dividend Growth Rate

In [133]:
def compute_cost_of_equity(ticker, dict_companies, average_market_return, date):
    
    #Using the CAPM model

    ten_year_bond_rate = pd.read_csv(filepath_or_buffer='10-year-treasury-bond-rate-yield-chart.csv', index_col=0)
    risk_free_rate = ten_year_bond_rate.loc[ten_year_bond_rate.index.str.contains(date)].iloc[0].value / 100

    beta = dict_companies[ticker]['_Company__profile'].loc['beta'][0]

    #We'll take the average return of the market from the average return of S&P500 which is 10,5% but being conservative, we'll leave it at 10%
    average_market_return = 0.1

    cost_of_equity = risk_free_rate+beta*(average_market_return-risk_free_rate)
    
    return cost_of_equity

In [134]:
def compute_dividend_growth_rate(ticker, dict_companies, disp):

    key_metrics = dict_companies[ticker]['_Company__key_metrics']
    dividend_growth_rate = key_metrics.loc['roe', key_metrics.columns[4*disp:4+(4*disp)]].sum() * (1-key_metrics.loc['payoutRatio', key_metrics.columns[4*disp:4+(4*disp)]].mean())
    
    return dividend_growth_rate

In [135]:
def compute_dividend_per_share(ticker, dict_companies, dividend_growth_rate, date):

    #Get dividends of the last year
    #Note: We'll take ttm dividends paid until the date in our key_metrics data
        
    dividends = dict_companies[ticker]['_Company__dividends']
    if isinstance(dividends, pd.DataFrame):
        dividends.index = pd.to_datetime(dividends.index, format='%Y-%m-%d')
        format_date = pd.to_datetime(date+'-01') 
        comparing_dates_arr = dividends.index < format_date
        if not any(list(comparing_dates_arr)):
            dividend_per_share = 0
            return dividend_per_share
        dividends_last_year = dividends.loc[dividends.index.year 
                                            == dividends.iloc[[np.where(comparing_dates_arr==True)[0][0]]].index.year[0]].dividend.sum()
        dividend_per_share = dividends_last_year * (1 + dividend_growth_rate)
    else:
        dividend_per_share = 0

    return dividend_per_share

In [136]:
def compute_intrinsic_value_ggm(ticker, dict_companies, average_market_return=0.1, year_disp=0):
    
    try:
        key_metrics = dict_companies[ticker]['_Company__key_metrics']
        date = key_metrics.columns[4*year_disp]
        check_all_quarters = key_metrics.columns[4*year_disp:4+(4*year_disp)]
    except:
        return "Error. No data for this ticker"
    
    cost_of_equity = compute_cost_of_equity(ticker, dict_companies, average_market_return, date)
    dividend_growth_rate = compute_dividend_growth_rate(ticker, dict_companies, year_disp)
    dividend_per_share = compute_dividend_per_share(ticker, dict_companies, dividend_growth_rate, date)
    
    intrinsic_value = dividend_per_share / (cost_of_equity - dividend_growth_rate)
    
    return intrinsic_value, date

In [137]:
def stock_price_intrinsic_value_comparator(dict_companies, ticker, intrinsic_value, date):
    
    stock_data = fa.stock_data(ticker, interval="1d")
    stock_data.index = pd.to_datetime(stock_data.index)
    
    format_date = pd.to_datetime(date+'-01')
    stock_price = stock_data.iloc[stock_data.index.get_loc(format_date, method='nearest')].close
    
    if stock_price == 0:
        pr_intrinsic_price = 0
    else:
        pr_intrinsic_price = intrinsic_value/stock_price 
    
    return pr_intrinsic_price

In [ ]:
ggm_by_sector = dict()
sectors = utils.get_sectors_and_industries(df, dict_companies)[0]
for sector in sectors:
    sector_df = utils.get_sector_companies(df, dict_companies, sector)
    ggm_by_sector[sector] = dict()
    for ticker in sector_df.index:
        #We'll take the average return of the market from the average return of S&P500 which is 10,5% but being conservative, we'll leave it at 10%
        intrinsic = compute_intrinsic_value_ggm(ticker, dict_companies, average_market_return=0.1, year_disp=(X years of disp))
        if intrinsic == "Error. No data for this ticker":
            ggm_by_sector[sector][ticker] = [np.nan]
            ggm_by_sector[sector][ticker].append(np.nan)
        else:
            ggm_by_sector[sector][ticker] = [intrinsic[0]]
            ggm_by_sector[sector][ticker].append(stock_price_intrinsic_value_comparator(dict_companies, ticker, intrinsic[0], intrinsic[1]))

In [ ]:
ggm_by_sector.keys()

In [ ]:
#Save dictionary
with open(f'models_outputs/GGM_output/X_year_disp/ggm_by_sector_Xyear.pickle', 'wb') as file:
    pickle.dump(ggm_by_sector, file)

**Retrieve the saved dictionary**

In [ ]:
ggm_by_sector_dict = dict()
with open('models_outputs/GGM_output/X_year_disp/ggm_by_sector_Xyear.pickle','rb') as file:
    raw_data = file.read()
    ggm_by_sector_dict.update(pickle.loads(raw_data))

In [ ]:
ggm_by_sector = dict()
for sector in ggm_by_sector_dict.keys():
    ggm_by_sector[sector] = pd.DataFrame.from_dict(ggm_by_sector_dict[sector], orient='index', columns=['intrinsic_value', 'intrval_vs_currentprice'])

In [ ]:
ggm_by_sector['Industrials']

## TESTING AN ALTERNATIVE WAY THAT ALSO WORKED
(It is not completely refined)

In [ ]:
#Using the CAPM model to compute the cost of equity

ten_year_bond_rate = pd.read_csv('10-year-treasury-bond-rate-yield-chart.csv', index_col=0)
key_metrics = dict_companies[ticker]['_Company__key_metrics']
mrq_date = key_metrics.columns[0]
risk_free_rate = ten_year_bond_rate.loc[ten_year_bond_rate.index.str.contains(mrq_date)].iloc[0].value / 100

beta = dict_companies[ticker]['_Company__profile'].loc['beta'][0]

#We'll take the average return of the market from the average return of S&P500 which is 10,5% but being conservative, we'll leave it at 10%
average_market_return = 0.1

cost_of_equity = risk_free_rate+beta*(average_market_return-risk_free_rate)
cost_of_equity

In [ ]:
dividend_growth_rate = key_metrics.loc['roe', key_metrics.columns[:4]].sum() * (1-key_metrics.loc['payoutRatio', key_metrics.columns[:4]].mean())
dividend_growth_rate   

In [ ]:
stock_data = fa.stock_data(ticker, interval="1d")
stock_data.index = pd.to_datetime(stock_data.index)
format_date = pd.to_datetime(mrq_date+'-01')
stock_price = stock_data.iloc[stock_data.index.get_loc(format_date, method='nearest')].close

dividend_per_share = stock_price * key_metrics.loc['dividendYield'][0]
dividend_per_share

In [ ]:
#Get dividends of the last year
#Note: We'll take ttm dividends paid untilt the mrq date in our key_metrics data

dividends = dict_companies[ticker]['_Company__dividends']
dividends.index = pd.to_datetime(dividends.index, format='%Y-%m-%d')
format_mrq_date = pd.to_datetime(mrq_date+'-01') 
comparing_dates_arr = dividends.index < format_mrq_date
dividends_last_year = dividends.loc[dividends.index.year 
                                    == dividends.iloc[[np.where(comparing_dates_arr==True)[0][0]]].index.year[0]].dividend.sum()
dividend_per_share = dividends_last_year * dividend_growth_rate
dividend_per_share

In [ ]:
intrinsic_value = dividend_per_share / (cost_of_equity - dividend_growth_rate)
intrinsic_value